# Train all text Data Set

In [1]:
# make an iterator that reads the file one line at a time instead of reading everything in memory at once
import os
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding='utf-8'):
                yield line.split()

In [2]:
# read all txt files from the given directory
#sentences = MySentences('C:/Thesis/Data/save/Master_Data/MD_lemmatized_ref_auth/') # a memory-friendly iterator
sentences = MySentences('C:/Thesis/Data/save/Master_Data/MD_lemmatized_latest/') # a memory-friendly iterator
#print(list(sentences))

In [3]:
#Create list to store the sentences
list_sent = []
temp = [[ list_sent.append([ w for w in word.split(',') if (len(w) > 2)]) for word in s] # filtered out the short words here
              for s in sentences]

In [ ]:
#from textblob import TextBlob 
#from termcolor import colored, cprint 
#for i in list_sent:
    
  #  b = TextBlob(i) 
#print(b)
# prints the corrected spelling 
#print (str(b.correct()))

# Get the word frequency of list sent

In [4]:
def getWordFreq(corpus):
    result = {}
    for data in corpus:
        for word in data:
            if word in result:
                result[word] += 1 #adding result in the dictionary
            else:
                result[word] = 1

    return result.items() #returning items

#print(getWordFreq(sentences))

In [5]:
fdist1 = getWordFreq(list_sent)

In [6]:
# convert dict_items to df
import pandas as pd
df_fdist = pd.DataFrame.from_dict(fdist1)

In [7]:
df_fdist = df_fdist.sort_values(by=[1], ascending=False)
df = df_fdist[0]

In [8]:
total = df_fdist[1].sum()
print(total) # almost 34 million

33800939


In [31]:
# set the threshold to remove the certain section of vocabulary
    
df_threshold = df_fdist[df_fdist[1].cumsum()/df_fdist[1].sum()<0.99]
df_threshold

,0,1
1668,employee,239477
129,effect,209724
1498,performance,207136
179,level,202324
80,study,199583
...,...,...
50503,nfo,13
17202,gerbing,13
20542,marvin,13
25497,baa,13


In [ ]:
#from textblob import TextBlob 
#df_fdist[1].apply(lambda txt: ''.join(TextBlob(txt).correct()))
#df_fdist['text'] = df_fdist[0].apply(lambda text: TextBlob(text).correct())

In [ ]:
minValue = df_threshold[1].min()
print(minValue)
print(len(df_threshold))

# K-means Clustering
Elbow method to determine the optimal number of k

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.cluster import KMeans

In [ ]:
df9['Cleaned_PaperText'] = pd.Series(texts, index = df9.index)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df)

In [ ]:
print(X)

In [ ]:
from scipy.spatial.distance import cdist

cluster_range = range( 1, 100 )
cluster_errors = []
for num_clusters in cluster_range:
    clusters = KMeans( num_clusters )
    clusters.fit( X )
    cluster_errors.append( clusters.inertia_ )
clusters_df = pd.DataFrame( { "num_clusters":cluster_range, "cluster_errors": cluster_errors } )

plt.figure(figsize=(12,6))
plt.plot( clusters_df.num_clusters, clusters_df.cluster_errors, marker = "o" )

# trained with best parameters

In [65]:
# initialize and train model {'min_count': 48, 'size': 100, 'window': 8, 'epochs': 50}
from gensim.models import Word2Vec
epochs=200
sentences = list_sent
model_bestpara = Word2Vec(
        sentences, # our dataset
        size=100, # this is the length of the vector to numerically represent the "meaning" of words
        window=15, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=50, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word


In [66]:
print(model_bestpara)

Word2Vec(vocab=14148, size=100, alpha=0.005)


In [70]:

import pandas as pd
pd.DataFrame(model_bestpara.wv.most_similar(positive = ['dedication'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,insert,0.411185
1,audiovisual,0.363597
2,section,0.363573
3,cfd,0.358088
4,axis,0.343598
5,nexis,0.343249
6,skew,0.341482
7,decomposition,0.340905
8,concise,0.334166
9,summarizes,0.330738


In [ ]:
# save the word2vec model {'min_count': 48, 'size': 100, 'window': 8, 'epochs': 50}-- model1
model_bestpara.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_latest/model_bestpara.model') 

In [ ]:
# initialize and train model with same parameters as of 6 articles.
# save the word2vec model with epoch-200, size-100, window-10, min_count-70, alpha-0.005
#******************************
from gensim.models import Word2Vec
import time
start_time = time.time()

epochs=200
sentences = list_sent
model_modified = Word2Vec(
        sentences, # our dataset
        size=100, # this is the length of the vector to numerically represent the "meaning" of words
        window=10, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=70, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word
#model1.train(sentences, total_examples=len(sentences), epochs=200) # epochs is the number of iterations over the full data set to train the model
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print(model_modified)

In [ ]:
import pandas as pd
pd.DataFrame(model_modified.wv.most_similar(positive = ['hrm'], topn=10), columns = ['word', 'similarity'])

In [ ]:
# save the word2vec model with epoch-200, size-100, window-10, min_count-70, alpha-0.005  -- model:2
model_modified.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_latest/model_modified.model') 

# Evaluation based on the observation table:

In [71]:
# Model-1
# initialize and train model {'min_count': 50, 'size':75 , 'window': 7, 'epochs': 50}
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=50
sentences = list_sent
model_obser_1 = Word2Vec(
        sentences, # our dataset
        size=75, # this is the length of the vector to numerically represent the "meaning" of words
        window=7, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=50, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word
print("--- %s seconds ---" % (time.time() - start_time))

--- 27.283753633499146 seconds ---


In [73]:
import pandas as pd
pd.DataFrame(model_obser_1.wv.most_similar(positive = ['engagement'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,energy,0.977402
1,allocation,0.976516
2,demand,0.968174
3,effort,0.967755
4,motivates,0.967653
5,threaten,0.966332
6,consume,0.965459
7,involvement,0.963354
8,encourage,0.960911
9,consumes,0.960707


In [ ]:
# save the word2vec model with {'min_count': 50, 'size':75 , 'window': 7, 'epochs': 50} observation 1
model_obser_1.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_latest/model_obser_1.model') 

In [74]:
# model-2
# initialize and train model {'min_count': 70, 'size':100 , 'window': 15, 'epochs': 200}
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=200
sentences = list_sent
model_obser_2 = Word2Vec(
        sentences, # our dataset
        size=100, # this is the length of the vector to numerically represent the "meaning" of words
        window=15, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=70, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word
print("--- %s seconds ---" % (time.time() - start_time))

--- 70.70103406906128 seconds ---


In [94]:
import pandas as pd
pd.DataFrame(model_obser_2.wv.most_similar(positive = ['engagement'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,enable,0.527002
1,effort,0.505735
2,persistent,0.496096
3,involvement,0.487881
4,stimulate,0.484856
5,library,0.471846
6,enhance,0.470841
7,adaptivity,0.469507
8,worksite,0.468392
9,implementation,0.467108


In [ ]:
# save the word2vec model with {'min_count': 70, 'size':100 , 'window': 15, 'epochs': 200}observation 2
model_obser_2.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_latest/model_obser_2.model') 

In [96]:
# Model-3
# initialize and train model {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300}
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=200
sentences = list_sent
model_obser_3 = Word2Vec(
        sentences, # our dataset
        size=150, # this is the length of the vector to numerically represent the "meaning" of words
        window=15, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=50, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word

print("--- %s seconds ---" % (time.time() - start_time))

--- 52.922818422317505 seconds ---


In [97]:
print(model_obser_3)

Word2Vec(vocab=14148, size=150, alpha=0.005)


In [102]:
import pandas as pd
pd.DataFrame(model_obser_3.wv.most_similar(positive = ['diversity'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,synergy,0.752372
1,programme,0.732433
2,discrimination,0.668144
3,asset,0.618216
4,calculates,0.599232
5,prohibits,0.597760
6,voluntary,0.574030
7,equip,0.569700
8,minority,0.560853
9,multicultural,0.558599


In [ ]:
# save the word2vec model with {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300} observation 3
model_obser_3.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_latest/model_obser_3.model') 

# Latest updated model- 3 observations freezing all the hyperparameters except window size

In [103]:
#Model 1
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=200
sentences = list_sent
model_1 = Word2Vec(
        sentences, # our dataset
        size=150, # this is the length of the vector to numerically represent the "meaning" of words
        window=8, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=50, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word

print("--- %s seconds ---" % (time.time() - start_time))

--- 46.44149398803711 seconds ---


In [104]:
import pandas as pd
pd.DataFrame(model_1.wv.most_similar(positive = ['diversity'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,synergy,0.780414
1,programme,0.734479
2,teenage,0.684074
3,multicultural,0.645586
4,discrimination,0.639988
5,impactful,0.622932
6,prohibits,0.596265
7,asset,0.591436
8,csr,0.590010
9,perspective,0.573124


In [111]:
model_1.save('C:/Thesis/Data/save/Master_Data/Model/word2vecWithDifferentWS/model_1.model') 

In [105]:
# Model-2
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=200
sentences = list_sent
model_2 = Word2Vec(
        sentences, # our dataset
        size=150, # this is the length of the vector to numerically represent the "meaning" of words
        window=12, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=50, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word

print("--- %s seconds ---" % (time.time() - start_time))

--- 54.83220028877258 seconds ---


In [106]:
import pandas as pd
pd.DataFrame(model_2.wv.most_similar(positive = ['diversity'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,synergy,0.778583
1,programme,0.740670
2,discrimination,0.662560
3,asset,0.612744
4,prohibits,0.608484
5,multicultural,0.598837
6,teenage,0.594525
7,equip,0.579206
8,elimination,0.573388
9,voluntary,0.566649


In [112]:
model_2.save('C:/Thesis/Data/save/Master_Data/Model/word2vecWithDifferentWS/model_2.model') 

In [107]:
#Model-3
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=200
sentences = list_sent
model_3 = Word2Vec(
        sentences, # our dataset
        size=150, # this is the length of the vector to numerically represent the "meaning" of words
        window=16, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=50, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word

print("--- %s seconds ---" % (time.time() - start_time))

--- 54.96534776687622 seconds ---


In [109]:
import pandas as pd
pd.DataFrame(model_3.wv.most_similar(positive = ['engagement'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,enable,0.527047
1,persistent,0.520092
2,effort,0.511320
3,library,0.503256
4,surveillance,0.491080
5,adaptivity,0.489512
6,involvement,0.485876
7,recovery,0.480000
8,adepartment,0.475325
9,implementation,0.474197


In [113]:
model_3.save('C:/Thesis/Data/save/Master_Data/Model/word2vecWithDifferentWS/model_3.model') 

# Train with Fasttext

# Model-1

In [ ]:
#{'min_count': 50, 'size':75 , 'window': 7, 'epochs': 50}

from gensim.models.fasttext import FastText
import time
start_time = time.time()
model_fasttext = FastText(size=75, window=7, min_count=50, sentences=list_sent, iter=50)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import pandas as pd
pd.DataFrame(model_fasttext.wv.most_similar(positive = ['turnover'], topn=10), columns = ['word', 'similarity'])

In [ ]:
# save the fasttext model with {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300}observation 1
model_fasttext.save('C:/Thesis/Data/save/Master_Data/Model/fasttext_latest/model_fasttext_1.model') 

# Model-2

In [ ]:

# {'min_count': 70, 'size':100 , 'window': 15, 'epochs': 200}

from gensim.models.fasttext import FastText
import time
start_time = time.time()

model_fasttext_2 = FastText(size=100, window=15, min_count=70, sentences=list_sent, iter=200)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import pandas as pd
pd.DataFrame(model_fasttext.wv.most_similar(positive = ['engagement'], topn=10), columns = ['word', 'similarity'])

In [ ]:
# save the fasttext model with {'min_count': 70, 'size':100 , 'window': 15, 'epochs': 200} observation 2
model_fasttext_2.save('C:/Thesis/Data/save/Master_Data/Model/fasttext_latest/model_fasttext_2.model') 

# Model-3

In [ ]:
# {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300}

from gensim.models.fasttext import FastText
import time
start_time = time.time()

model_fasttext_3 = FastText(size=150, window=20, min_count=100, sentences=list_sent, iter=300)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import pandas as pd
pd.DataFrame(model_fasttext.wv.most_similar(positive = ['engagement'], topn=10), columns = ['word', 'similarity'])

In [ ]:
# save the fasttext model with {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300} observation 3
model_fasttext_3.save('C:/Thesis/Data/save/Master_Data/Model/fasttext_latest/model_fasttext_3.model') 

# Evaluation  for primary words diversity, empowerment, engagement and climate based on different vector dimension size

# Word2Vec

In [4]:
# Model-3 remain the standard model but with different vector dimentsion size
# For Size = 75
# initialize and train model {'min_count': 100, 'size':75 , 'window': 20, 'epochs': 300}
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=300
sentences = list_sent
model_obser_75 = Word2Vec(
        sentences, # our dataset
        size=75, # this is the length of the vector to numerically represent the "meaning" of words
        window=20, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=100, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word

print("--- %s seconds ---" % (time.time() - start_time))

--- 94.2230167388916 seconds ---


In [9]:
import pandas as pd
pd.DataFrame(model_obser_75.wv.most_similar(positive = ['climate'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,adaptivity,0.567759
1,invaluable,0.519621
2,domaing,0.515985
3,multicultural,0.479426
4,chair,0.479384
5,champion,0.473263
6,oregon,0.469937
7,periodical,0.469288
8,portland,0.462311
9,spiritual,0.455308


In [5]:
# save the word2vec model with {'min_count': 100, 'size':75 , 'window': 20, 'epochs': 300} observation 3
model_obser_75.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_different_size/model_obser_75.model') 

In [6]:
# Model-3 remain the standard model but with different vector dimentsion size
# For Size = 100
# initialize and train model {'min_count': 100, 'size':75 , 'window': 20, 'epochs': 300}
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=300
sentences = list_sent
model_obser_100 = Word2Vec(
        sentences, # our dataset
        size=100, # this is the length of the vector to numerically represent the "meaning" of words
        window=20, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=100, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word

print("--- %s seconds ---" % (time.time() - start_time))

--- 99.96387577056885 seconds ---


In [14]:
import pandas as pd
pd.DataFrame(model_obser_100.wv.most_similar(positive = ['climate'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,adaptivity,0.561197
1,domaing,0.509782
2,invaluable,0.495539
3,multicultural,0.490148
4,chair,0.455568
5,champion,0.455383
6,workfamily,0.450441
7,portland,0.448808
8,oregon,0.447884
9,mentor,0.443087


In [7]:
# save the word2vec model with {'min_count': 100, 'size':100 , 'window': 20, 'epochs': 300} observation 3
model_obser_100.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_different_size/model_obser_100.model') 

In [8]:
# Model-3 remain the standard model but with different vector dimentsion size
# For Size = 150
# initialize and train model {'min_count': 100, 'size':75 , 'window': 20, 'epochs': 300}
from gensim.models import Word2Vec
import time

start_time = time.time()
epochs=300
sentences = list_sent
model_obser_150 = Word2Vec(
        sentences, # our dataset
        size=150, # this is the length of the vector to numerically represent the "meaning" of words
        window=20, # this is the number of neighboring words to consider when assigning "meaning" to a word
        min_count=100, # minimum number of occurrences
        alpha = 0.005,
        iter =  epochs) # this is how fast the model adapts its representation of the "meaning" of a word

print("--- %s seconds ---" % (time.time() - start_time))

--- 106.18697094917297 seconds ---


In [19]:
import pandas as pd
pd.DataFrame(model_obser_150.wv.most_similar(positive = ['climate'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,adaptivity,0.555440
1,domaing,0.519504
2,invaluable,0.482566
3,multicultural,0.478398
4,chair,0.469153
5,champion,0.465655
6,periodical,0.436557
7,mentor,0.435273
8,oregon,0.426130
9,portland,0.425120


In [9]:
# save the word2vec model with {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300} observation 3
model_obser_150.save('C:/Thesis/Data/save/Master_Data/Model/word2vec_different_size/model_obser_150.model') 

# FastText 

In [10]:
# {'min_count': 100, 'size':75 , 'window': 20, 'epochs': 300}

from gensim.models.fasttext import FastText
import time
start_time = time.time()

model_fasttext_75 = FastText(size=75, window=20, min_count=100, sentences=list_sent, iter=300)

print("--- %s seconds ---" % (time.time() - start_time))

--- 1196.5456109046936 seconds ---


In [24]:
import pandas as pd
pd.DataFrame(model_fasttext_75.wv.most_similar(positive = ['climate'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,climateg,0.947593
1,climb,0.793193
2,animate,0.702181
3,clip,0.578999
4,intimate,0.507553
5,ultimate,0.495731
6,mate,0.481111
7,teammate,0.457302
8,approximate,0.439182
9,cliff,0.428637


In [11]:
# save the fasttext model with {'min_count': 100, 'size':75 , 'window': 20, 'epochs': 300} observation 3
model_fasttext_75.save('C:/Thesis/Data/save/Master_Data/Model/fasttext_different_size/model_fasttext_75.model') 

In [12]:
# {'min_count': 100, 'size':100 , 'window': 20, 'epochs': 300}

from gensim.models.fasttext import FastText
import time
start_time = time.time()

model_fasttext_100 = FastText(size=100, window=20, min_count=100, sentences=list_sent, iter=300)

print("--- %s seconds ---" % (time.time() - start_time))

--- 976.479766368866 seconds ---


In [29]:
import pandas as pd
pd.DataFrame(model_fasttext_100.wv.most_similar(positive = ['climate'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,climateg,0.949268
1,climb,0.773934
2,animate,0.700782
3,clip,0.535080
4,intimate,0.500857
5,teammate,0.474088
6,ultimate,0.464241
7,mate,0.445071
8,cliff,0.412281
9,approximate,0.409533


In [13]:
# save the fasttext model with {'min_count': 100, 'size':100 , 'window': 20, 'epochs': 300} observation 3
model_fasttext_100.save('C:/Thesis/Data/save/Master_Data/Model/fasttext_different_size/model_fasttext_100.model') 

In [14]:
# {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300}

from gensim.models.fasttext import FastText
import time
start_time = time.time()

model_fasttext_150 = FastText(size=150, window=20, min_count=100, sentences=list_sent, iter=300)

print("--- %s seconds ---" % (time.time() - start_time))

--- 39530.89315152168 seconds ---


In [34]:
import pandas as pd
pd.DataFrame(model_fasttext_150.wv.most_similar(positive = ['climate'], topn=10), columns = ['word', 'similarity'])

,word,similarity
0,climateg,0.940577
1,climb,0.782772
2,animate,0.702102
3,clip,0.569480
4,intimate,0.558011
5,mate,0.531406
6,ultimate,0.525396
7,teammate,0.468850
8,proximate,0.429999
9,cliff,0.407638


In [15]:
# save the fasttext model with {'min_count': 100, 'size':150 , 'window': 20, 'epochs': 300} observation 3
model_fasttext_150.save('C:/Thesis/Data/save/Master_Data/Model/fasttext_different_size/model_fasttext_150.model') 

In [ ]:
#*********************************************************************************************

In [ ]:
# save the word2vec model
model_fasttext.save('C:/Thesis/Data/save/Master_Data/Model/paraModified/fasttext/save_fasttext_model_12_articles.model')

In [ ]:
# sg- either 1 or 0. 1 means to train a skip-gram model and 0 means to train a CBOW model.
# hs- either 1 or 0. if this is 1, ten hierarchical softmax will be used as the loss function.
# workers- number of threads for training
# iter- number of iterations over the samples
# size- the dimensions of the word vectors and hence must be an integer.

from gensim.models.fasttext import FastText
model_fasttext = FastText(list_sent, sg=1, hs=1, size=100, workers=12, iter=200, min_count=81)

In [ ]:
import pandas as pd
pd.DataFrame(model_fasttext.wv.most_similar(positive = ['leadership'], topn=10), columns = ['word', 'similarity'])

In [ ]:
import pandas as pd
# get an example from the model: find single terms that are alike [positive] or dissimilar [negative] to specified reference words
pd.DataFrame(model_fasttext.wv.most_similar(positive = ['turnover'], topn=10), columns = ['word', 'similarity'])

In [ ]:
import pandas as pd
# get an example from the model: find single terms that are alike [positive] or dissimilar [negative] to specified reference words
pd.DataFrame(model_fasttext.wv.most_similar(positive = ['selection'], topn=20), columns = ['word', 'similarity'])

In [ ]:
# save the word2vec model
model_fasttext.save('C:/Thesis/Data/save/Master_Data/Model/save_fasttext_model_12_articles.model')

# Train with Glove 

In [ ]:
#from glove import Corpus, Glove
from glove import *

In [ ]:


#Creating a corpus object
corpus = Corpus() 

#Training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lines, window=10)

glove = Glove(no_components=5, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
#glove.save('glove.model')

# Evaluation : Wordsim353
wordsim is a way of measuring how a model captures similarity, rather than relatedness or association.

In [ ]:
from gensim.test.utils import datapath
similarities = model.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [ ]:
similarities

# Simlex999
Measuring how a model captures the relateness.

In [ ]:
related = model.wv.evaluate_word_pairs(datapath('simlex999.txt'))   

In [ ]:
related

In [ ]:
# Select the ones you want
df1 = df[['word']]
t = len(df1['word'])
if t < 4:
    print("yees")
else:
    print("noo")

In [ ]:
df_test = pd.DataFrame({'a': [1, 10, 8, 11, -1],
                       'b': list('abdce'),
                       'c': [1.0, 2.0, np.nan, 3.0, 4.0]})
df_test.nlargest(3, 'a')

In [ ]:
#[df.nlargest(3,'similarity'),'word']= 'background-color: yellow'

In [ ]:
# Using DataFrame.drop
df.drop(df.columns[[1, 2]], axis=1, inplace=True)

# drop by Name
df1 = df1.drop(['B', 'C'], axis=1)

# Select the ones you want
df1 = df[['a','d']]

In [ ]:
import numpy as np
labels = []
count = 0
max_count = 50
X = np.zeros(shape=(max_count,len(model['model'])))

for term in model.wv.vocab:
    X[count] = model[term]
    labels.append(term)
    count+= 1
    if count >= max_count: break

# It is recommended to use PCA first to reduce to ~50 dimensions
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
X_50 = pca.fit_transform(X)

# Using TSNE to further reduce to 2 dimensions
from sklearn.manifold import TSNE
model_tsne = TSNE(n_components=2, random_state=0)
Y = model_tsne.fit_transform(X_50)

# Show the scatter plot
import matplotlib.pyplot as plt
plt.scatter(Y[:,0], Y[:,1], 20)



# Add labels
for label, x, y in zip(labels, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy = (x,y), xytext = (0, 0), textcoords = 'offset points', size = 10)

#plt.show()

In [ ]:
# save the model
model_fasttext.save('C:/Thesis/Data/save/Master_Data/Model/fastText/ft_model_with_6_articles.model') 

# Highlight Specific Number


In [ ]:
import pandas as pd
# get an example from the model: find single terms that are alike [positive] or dissimilar [negative] to specified reference words
df = pd.DataFrame(model.wv.most_similar(positive = ['leadership'], topn=10), columns = ['word', 'similarity'])

In [ ]:
def select_col(x):
    c1 = 'background-color: yellow'
    c3 = 'background-color: #FFFF9E'
    c2 = ''
    temp = x['similarity']>0.6
    temp2 = (x['similarity'] < 0.6)
    
    df1 = pd.DataFrame(c2, index=df.index, columns = df.columns)
    
    df1.loc[temp,'word'] = c1
    df1.loc[temp2,'word'] = c3
    
    return df1
df.style.apply(select_col,axis=None)

In [ ]:
b